In [3]:
# Important libraries and modules

import numpy as np
from itertools import permutations
import gzip


from qiskit import*

import time

from qiskit.aqua.algorithms import VQE
from qiskit.aqua.algorithms import QAOA

from qiskit.aqua.components.optimizers import SPSA

from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.optimizers import COBYLA

from qiskit import IBMQ

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit

from qiskit.optimization.applications.ising.tsp import TspData
import qiskit.optimization.applications.ising.tsp as tsp
from qiskit.circuit.library import TwoLocal
from qiskit.circuit.library import RealAmplitudes

ImportError: cannot import name 'BaseBackend' from 'qiskit.providers' (c:\Users\quint\miniconda3\envs\quantum2\lib\site-packages\qiskit\providers\__init__.py)

In [ ]:
# # Load account from disk
IBMQ.load_account()
IBMQ.providers()

In [13]:
def readInData():
    
    """
    Output G : N by N distance matrix from Matrices1a.txt file. 
    
    """
    
    G = []

    p = [3, 4,5,6,7,8,9,10,11]
    q = [i**(2) for i in p ]
    m = 0
    
    v = open("Matrices.txt"  , "r")
    w = v.read().split()
    for i in range (len(w)):
        w[i] = int(float(w[i]))
    for i in range (len(q)):
        G.append(np.reshape(w[m:m+q[i]] , (p[i] , p[i])))
        m = m + q[i]
        
    return G

distanceMatrix =  readInData() #Array of different sized matrices

In [ ]:
def determineIfFeasible(result):
    
    """
    Determines if eigenstate is feasible or infeasible.
    
    Output: arr = Infeasible if eiegenstate is infeasible or arr = binary array of feasible solution
    
    """
    
    data = sorted(result['eigenstate'].items(), key=lambda item: item[1])[::-1]
    for i in range(len(data)):
        a = tsp.tsp_feasible(data[i][0])
        arr = 'Infeasible'
        if a == True:
            b = str(data[i][0])
            arr = [b , data[i][1]]
            break
            
    return arr

In [14]:
def optimal(a,b,c,f,u):
    
    """
    Read in data of initial optimal point that will be used in the quantum algorithm
    
    """
    openfile = open("optimal.txt" , "r")
    readFile = openfile.read().split()
    t = []
    for i in readFile:
        if i != ',':
            q = len(i)
            t.append(float(i[0:q-1]))

    v, r, o, d, z = np.array(t[0:a]), np.array(t[a:a+b]), np.array(t[a+b : a+b+c]), np.array(t[a+b+c:a+b+c+f]), np.array(t[a+b+c+f:a+b+c+f+u])
    
    return [v,r,o,d,z]

R = optimal(54,96,100,216,294) #Array of corresponding initial points

In [ ]:
def variationalQuantumEigensolver(numIter,numShots,distanceMatrix, varForm, initialPoint, deviceName):
    
    """
    Implementation of the VQE
    
    Output: classial TSP solution (total length of tour) and time taken to execute algorithm
    
    """
    # Mappining of problem to ising hamiltonian
    x = TspData('tmp',len(matrix),np.zeros((3,3)),distanceMatrix)
    qubitOp ,offset = tsp.get_operator(x)
    
    seed = 10598
    
    # Generate a circuit
    spsa = SPSA(maxiter = numIter)
        
    if varForm == 'vf1':
        ry = RealAmplitudes(qubitOp.num_qubits, entanglement='linear')
        
    elif varForm == 'vf2':
        ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
    
    
    
    runVqe =  VQE(qubitOp, ry, spsa,include_custom=True, initial_point = initialPoint)    
    
    
    my_provider = IBMQ.get_provider(ibm_hub) #Replace ibm_hub with appropriate qiskit hub name
    
    device = my_provider.get_backend(deviceName) # deviceName is the Device of IBM quantum device in a string
    
    quantum_instance = QuantumInstance(device, seed_simulator=seed, seed_transpiler=seed,shots = numShots,
                                       skip_qobj_validation = False)
        
        
    result = runVqe.run(quantum_instance)
    
    #Convert quantum result into its classical form and determine if feasible or infeasible
    answer = determineIfFeasible(result)
    
    if answer == 'Infeasible':
        solution = -1
    else:
        binarry = [int(p) for p in answer[0]]
        route = tsp.get_tsp_solution(binarry)
        solution = tsp.tsp_value(route, distanceMatrix)
        
    return  solution, result['optimizer_time']

In [ ]:
## Example for 3 by 3 instance implemented using VQE:

numIter = 1
numShots = 1024
distanceMatrix = distanceMatrix[0] 
varForm = 'vf1' #vf1 indicates the RealAmplitude form 
deviceName = 'ibmq_cambridge' 
initialPoint = R[0]

finalResult = variationalQuantumEigensolver(numIter,numShots, distanceMatrix, varForm, initialPoint, deviceName)